# Prueba 2 de la prácticas de MN2. Integración y derivación numérica

Guardar el archivo con el formato ```Apellido1_Apellido2_Nombre_DNI.ipynb```.

**I. Derivación numérica.**<br>
1.   Calcular el polinomio de interpolación para los tres nodos $a-2h$, $a-h$, $a$.
2.   Calcular la derivada segunda regresiva de forma aproximada usando interpolación polinómica. Comprobar que la fórmula obtenida es distinta a la derivada segunda progresiva para los nodos $a,a+h,a+2h$ (ya hecho en el notebook de la prácticas).
3.   Calcular grado de exactitud para funciones de tipo polinomico, es decir, comparar la fórmula aproximada de derivada segunda con la derivada segunda exacta de la base $\{1,x,...,x^n\}$ de $\mathbb{P}_n$. Verificar que se cumple el teorema de la limitación del grado de exactitud (Teorema 2 de los apuntes extendidos).
4.   Probar la fórmula aproximada de la derivada segunda regresiva para la función $f(z)=\frac{z^2-2}{1+z^3}$ y para un paso $h=10^{-j}$ con $j=1,2,\ldots,20$. Comparar con el valor exacto de la derivada segunda obtenido con el método de sympy ```sp.diff()```. Comprobar para qué valor de $j$ deja de funcionar la fórmula aproximada y da 0.

**II. Integración numérica.**<br>
1.   Calcular la fórmula gaussiana de integración numérica para 3 nodos y un peso constante $w=1$ en el intervalo $[0,2]$.
2.   Aplicar la fórmula gaussiana obtenida a la función $f(x)=\cosh(x)-1$.
3.   Comparar el resultado del apartado anterior con el que sale integrando con ```sp.integrate()```. Utilizar el método ```.evalf()``` para obtener una expresión numérica de ambas integrales.


In [139]:
import numpy as np # Importamos el módulo NumPy con el pseudónimo np
import sympy as sp # Importamos el módulo SymPy con el pseudónimo sp
# sp.init_printing() # con esta opción obtendremos bonitas fórmulas en la pantalla
# pero en las últimas versiones de Python parece ser que no hace falta indicarlo

# **I) Derivación numérica**

## 1)

In [140]:
#Declaracion de variables simbolicas
f = sp.Function('f')# Crea función simbólica
a,h = sp.symbols('a,h')
z = sp.Symbol('z')  # utilizaremos z como variable simbólica

In [141]:
x = [a-2*h,a-h,a]  # estos serán los nodos de interpolación, 3 nodos -> polinomio interpolación de grado 2
y = [f(x[0]),f(x[1]),f(x[2])]  # y los valores interpolados

# Polinomio de grado 1
p0 = y[0]  # vamos a ir construyendo el polinomio de interpolación
p1 = p0 + (z-x[0])/(x[1]-x[0])*(y[1]-y[0]) # usando la idea de Newton

# Polinomio de grado 2
D = sp.Symbol('D')   # D será la correspondiente diferencia dividida:  D = f[x0,x1,x2]
p2 = p1 + (z-x[0])*(z-x[1])*D

# pero también tenemos aún que garantizar que p(x2) = y2
sol2=sp.solve(p2.subs({z:x[2]})-y[2],D)  # para ello resolvemos y despejamos el valor de D
#print("Diferencia dividida de segundo orden:\n",sol2,"\n")

D=sol2[0]
p2 = p1 + (z-x[0])*(z-x[1])*D

print("Polinomio de interpolacion de grado 2:\n")
p2

Polinomio de interpolacion de grado 2:



f(a - 2*h) + (-f(a - 2*h) + f(a - h))*(-a + 2*h + z)/h + (-a + h + z)*(-a + 2*h + z)*(f(a) + f(a - 2*h) - 2*f(a - h))/(2*h**2)

## 2)

In [142]:
#Hallamos formula derivando 2 veces p2 y evaluando en a
formula= sp.diff(p2,z,2).subs({z:a}).simplify()

print("\nFormula de aproximacion de segunda derivada (diferencia regresiva):\n")
formula


Formula de aproximacion de segunda derivada (diferencia regresiva):



(f(a) + f(a - 2*h) - 2*f(a - h))/h**2

In [143]:
#Veamos que es distinta a la de diferencia progresiva:

x = [a,a+h,a+2*h]  # estos serán los nodos de interpolación, 3 nodos -> polinomio interpolación de grado 2
y = [f(x[0]),f(x[1]),f(x[2])]  # y los valores interpolados

# Polinomio de grado 1
p0 = y[0]  # vamos a ir construyendo el polinomio de interpolación
p1 = p0 + (z-x[0])/(x[1]-x[0])*(y[1]-y[0]) # usando la idea de Newton

# Polinomio de grado 2
D = sp.Symbol('D')   # D será la correspondiente diferencia dividida:  D = f[x0,x1,x2]
p2 = p1 + (z-x[0])*(z-x[1])*D

# pero también tenemos aún que garantizar que p(x2) = y2
sol2=sp.solve(p2.subs({z:x[2]})-y[2],D)  # para ello resolvemos y despejamos el valor de D
#print("Diferencia dividida de segundo orden:\n",sol2,"\n")

D=sol2[0]
p2 = p1 + (z-x[0])*(z-x[1])*D

#Hallamos formula derivando 2 veces p2 y evaluando en a
formula= sp.diff(p2,z,2).subs({z:a}).simplify()

print("Formula de de aproximacion de segunda derivada (diferencia progresiva):\n")
formula

Formula de de aproximacion de segunda derivada (diferencia progresiva):



(f(a) - 2*f(a + h) + f(a + 2*h))/h**2

Vemos que ambas fórmulas son distintas.

## 3)

In [144]:
def formula(f):
  return (f(a)+f(a-2*h)-2*f(a-h))/h**2

# Derivada segunda de x^i con i=0,...,10
valoresexactosderivadasegunda =[sp.diff(z**i,z,2).subs({z:a}) for i in range(10)]
# Evaluamos formula() en x^i con i=0,...,10, es decir, derivamos x^i con la fórmula de derivación obtenida a partir de interpolación
valoresaproximadosderivadasegunda =[formula(lambda z:z**i).simplify() for i in range(10)]

# Estamos comparando el funcional derivada con el funcional calculado con la interpolación polinómica
np.array(valoresaproximadosderivadasegunda) - np.array(valoresexactosderivadasegunda)

array([0, 0, 0, -6*h, -24*a*h + 14*h**2,
       -20*a**3 + (a**5 + (a - 2*h)**5 - 2*(a - h)**5)/h**2,
       -30*a**4 + (a**6 + (a - 2*h)**6 - 2*(a - h)**6)/h**2,
       -42*a**5 + (a**7 + (a - 2*h)**7 - 2*(a - h)**7)/h**2,
       -56*a**6 + (a**8 + (a - 2*h)**8 - 2*(a - h)**8)/h**2,
       -72*a**7 + (a**9 + (a - 2*h)**9 - 2*(a - h)**9)/h**2], dtype=object)

Podemos comprobar que el grado de exactitud que se alcanza es 2. Además, por el teorema sabemos que a partir de n+k+1=5, con n=2, k=2, la fórmula ya no es exacta, donde n es el número de nodos -1 y k el orden de derivación. Por tanto, el grado de exactitud máximo es 4.

## 4)

In [145]:
def formula(f,a,h):
  return (f(a)+f(a-2*h)-2*f(a-h))/h**2

def f1(z):
  return (z**2-2)/(1+z**3)

a1=2
n=21
aproximaciones=[]
errores=[]
val_exacto=sp.diff(f1(z),z,2).subs({z:a1}).evalf()

print("Valor exacto: ", val_exacto)
print("j\t\t\tAproximación\t\t\t\tError")
print("------------\t\t------------\t\t\t\t-----------")

for i in range (1,n):
  aproximaciones.append(formula(f1,a1,10**-i))
  errores.append(abs(aproximaciones[i-1]-val_exacto))
  print("{}\t\t{}\t\t\t\t{}".format(i, aproximaciones[i-1], errores[i-1]))


Valor exacto:  -0.469135802469136
j			Aproximación				Error
------------		------------				-----------
1		-0.6000287330661435				0.130892930597008
2		-0.4806528060502524				0.0115170035811166
3		-0.470273181230052				0.00113737876091624
4		-0.4692493948965648				0.000113592427429032
5		-0.46914694351585234				0.0000111410467165673
6		-0.46918025020659115				0.0000444477374553776
7		-0.46074255521944				0.00839324724969576
8		-1.1102230246251563				0.641087222156021
9		0.0				0.469135802469136
10		0.0				0.469135802469136
11		0.0				0.469135802469136
12		0.0				0.469135802469136
13		-5551115123.125782				5551115122.65665
14		-555111512312.5782				555111512312.109
15		55511151231257.82				55511151231258.3
16		0.0				0.469135802469136
17		0.0				0.469135802469136
18		0.0				0.469135802469136
19		0.0				0.469135802469136
20		0.0				0.469135802469136


# **II) Integración numérica**

## 1)

In [146]:
def w(z):
  return 1

def Gaussiana_Peso_Simbolica(w,a,b,n):
  """
  Funcion que calcula mediante aproximacion gaussiana una integral.
  w: peso
  [a,b]: intervalo
  n: numero de nodos
  """
  grexact=2*n-1
  #Obtenemos lista de coeficientes
  p = sp.symbols('p0:'+ str(n)) # generamos una tupla de n nodos, ahora empezamos por p0
  nodos2 = list(p)

  c = sp.symbols('c0:'+ str(n)) # generamos una tupla de n coeficientes
  coefs2 = list(c)
  coefs2

  incogs2 = coefs2 + nodos2
  ################################################################################

  #Resolvemos ecuacion
  ecs2 = [np.dot([(z**i).subs({z:nodos2[j]}) for j in range(n)],coefs2)-sp.integrate(w(z)*z**i,(z,a,b)) for i in range(grexact+1)]
  solsGauss2 = sp.solve(ecs2,incogs2)
  ################################################################################

  #Obtenemos coeficientes y nodos
  for i in range(n):
      nodos2[i] = solsGauss2[0][n+i]
      coefs2[i] = solsGauss2[0][i]
  ################################################################################

  #Construimos formula
  formGauss2 = np.dot([f(nodos2[i]) for i in range(n)],coefs2)
  ################################################################################
  return formGauss2


def Gaussiana_Peso(f,w,a,b,n):
  """
  Funcion que calcula mediante aproximacion gaussiana una integral.
  f: funcion
  w: peso
  [a,b]: intervalo
  n: numero de nodos
  """
  grexact=2*n-1
  #Obtenemos lista de coeficientes
  p = sp.symbols('p0:'+ str(n)) # generamos una tupla de n nodos, ahora empezamos por p0
  nodos2 = list(p)

  c = sp.symbols('c0:'+ str(n)) # generamos una tupla de n coeficientes
  coefs2 = list(c)
  coefs2

  incogs2 = coefs2 + nodos2
  ################################################################################

  #Resolvemos ecuacion
  ecs2 = [np.dot([(z**i).subs({z:nodos2[j]}) for j in range(n)],coefs2)-sp.integrate(w(z)*z**i,(z,a,b)) for i in range(grexact+1)]
  solsGauss2 = sp.solve(ecs2,incogs2)
  ################################################################################

  #Obtenemos coeficientes y nodos
  for i in range(n):
      nodos2[i] = solsGauss2[0][n+i]
      coefs2[i] = solsGauss2[0][i]
  ################################################################################

  #Construimos formula
  formGauss2 = np.dot([f(nodos2[i]) for i in range(n)],coefs2)
  ################################################################################
  return formGauss2.evalf()

In [147]:
n=3
a1,b1=0,2

formula=Gaussiana_Peso_Simbolica(w,a1,b1,n)
formula

8*f(1)/9 + 5*f(1 - sqrt(15)/5)/9 + 5*f(sqrt(15)/5 + 1)/9

## 2)

In [148]:
def f2(z):
  return -1 + sp.cosh(z)

aproximacion=Gaussiana_Peso(f2,w,a1,b1,n)
aproximacion

1.62675939993726

## 3)

In [149]:
val_exacto=sp.integrate(f2(z)*w(z),(z,a1,b1)).evalf()
error=abs(aproximacion - val_exacto)
print("Valor exacto: {}\n".format(val_exacto))
print("n\tAproximación\t\tError")
print("-\t------------\t\t-----")
print("{}\t{}\t{}".format(n, aproximacion, error))

Valor exacto: 1.62686040784702

n	Aproximación		Error
-	------------		-----
3	1.62675939993726	0.000101007909756534
